# Store

> Provides storage for documents

In [ ]:
#| default_exp store

In [ ]:
#| export
from nbdev_rag.base import Document, abstractmethod, ABC
from typing import Union, List, Dict
from uuid import UUID

In [ ]:
class BaseDocumentStore(ABC):
    """
    Base class for document storage"""
    def __init__(self, documents : Dict[str, Document] = {}):
        pass
    @abstractmethod
    def __call__(self, db): #Connect to backend and specific collection parameters
        pass
    @abstractmethod    
    def add(self, document: Document, allow_duplicates = False):
        pass
    @abstractmethod 
    def ids(self):
        pass
    @abstractmethod    
    def delete(self, ids: Union[List, str]):
        pass
    @abstractmethod
    def get(self, ids: Union[UUID, List] = None):
        pass
        

In [ ]:
class DocumentStore(BaseDocumentStore):
    """Key value type document store that store documents by their id in a dictionary.
    Also checks for duplicates via hashing and doesn't admit them"""
    def __init__(self, documents : Dict[str, Document] = {}):
        self.documents = documents

    def __call__(self, db): #Connect to backend and specific collection parameters
        pass
        
    def add(self, document: Document, allow_duplicates = False):
        #Here check for docs with the same hash or id. If diff id and hash add
        for stored_document in self.documents:
            if self.documents[stored_document].hash == document.hash:
                if allow_duplicates == True:
                    self.documents[document.id] = document
                    return f"Document with duplicate hash {document.hash} has been added"
                return f"You tried to add a duplicate document: {document.hash}"
            elif self.documents[stored_document].id == document.id:
                self.documents[document.id] = document
                return f"Document with id {document.id} has been updated"
        self.documents[document.id] =  document

    def ids(self):
        doc_ids = [doc for doc in self.documents]
        return doc_ids

    def delete(self, ids: Union[List, str]):
        deleted_docs = []
        if isinstance(ids, List):
            for id in ids:
                deleted_doc = self.documents.pop(id, None)
                if deleted_doc != None:
                    deleted_docs.append(deleted_doc)
        elif isinstance(ids, UUID):
            deleted_doc = self.documents.pop(ids, None)
            print(f'Ids are: {ids} and theoretically deleted doc is {deleted_doc}')
            if deleted_doc != None:
                deleted_docs.append(deleted_doc)
        return f"The following docs have been deleted {deleted_docs}"
        
    def get(self, ids: Union[UUID, List] = None):
        if ids == None:
            ids = self.ids()
        if isinstance(ids, List):
            docs = [self.documents.get(id, None) for id in ids]
            return docs
        elif isinstance(ids, UUID):
            return self.documents.get(ids, None)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()